In [10]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, DatetimeTickFormatter
from bokeh.io import output_file, show
import pandas as pd
import numpy as np

from bokeh.plotting import figure

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (14,10)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
#Settings for plotting
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps


import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
locale.setlocale(locale.LC_ALL,"Danish")

# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

'Danish_Denmark.1252'

# Load data 

In [11]:
ssidatapath = "ssi_data"
rootdir = os.getcwd()[:-5] + ssidatapath
dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')
dfCase = pd.DataFrame(columns=dfAdm.columns)

In [12]:
for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfAdm.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    casedatadf = pd.read_csv(curFilePath,delimiter=';',dtype=str)

    casedatadf['Antal_bekræftede_COVID-19'] = pd.to_numeric(casedatadf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))

    dfCase.loc[curIndex] = casedatadf['Antal_bekræftede_COVID-19'].values

# Data: Collect the data in same style

In [13]:
# Same things, but with cases
dfCaseDiff = dfCase.diff().iloc[1:]

dfCaseDiff['Dato'] = curDates[1:]

# Bad hack to ignore 10-15 year olds!
dfCaseDiff['10-19'] = np.multiply(dfCaseDiff['10-19'],0.4)

dfCaseDiffSum = dfCaseDiff.iloc[:,1:-2].cumsum(axis=1)
dfCaseDiffSum['Dato'] = curDates[1:]

# dfCaseDiff = dfCaseDiff.iloc[:-1]
# dfCaseDiffSum = dfCaseDiffSum.iloc[:-1]


dfCaseDiffSum['I alt'] = dfCaseDiffSum['90+']
# dfCaseDiffSum

In [14]:
dfAdmDiff = dfAdm.diff().iloc[1:]

dfAdmDiff['Dato'] = curDates[1:]

dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
dfAdmDiffSum['Dato'] = curDates[1:]

# dfAdmDiff = dfAdmDiff.iloc[:-1]
# dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']

dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

Aldersgruppe,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Dato,I alt
17_11_2020,4,17,35,61,110,165,228,270,284,2020-11-17,284
24_11_2020,1,17,36,64,113,163,222,292,303,2020-11-24,303
01_12_2020,1,8,20,52,97,130,193,252,267,2020-12-01,267
08_12_2020,7,30,62,103,164,231,341,421,442,2020-12-08,442
15_12_2020,14,41,89,148,250,366,486,589,625,2020-12-15,625
22_12_2020,15,57,127,222,365,517,753,938,992,2020-12-22,992
29_12_2020,12,60,110,208,356,526,811,1023,1084,2020-12-29,1084
05_01_2021,14,51,109,181,302,463,674,898,971,2021-01-05,971
12_01_2021,6,39,76,125,233,349,555,766,824,2021-01-12,824
19_01_2021,3,18,50,96,183,280,447,599,656,2021-01-19,656


# Load calculations

In [15]:
datafilename = '../model_results_spread_prediction_var_vacc.csv'
df = pd.read_csv(datafilename)
df.End =  pd.to_datetime(df.End,format='%Y-%m-%d')

In [16]:
df['RT2'] = np.round(np.multiply(df.RT,10)).astype(int)

In [17]:
curdf = df[df.vacc1 == 0.75]

In [18]:
def cmaptohex(tup):
    (r,g,b,a) = tup
    return "#{:02x}{:02x}{:02x}".format(int(r*255),int(g*255),int(b*255))

In [19]:
curdf[curdf.Age == '10-19']

,Unnamed: 0,Age,CaseHospRate,Population,Start,End,RT,Count,CountCumul,vacc1,PctCumul,HospCumul,RT2
154,154,10-19,0.01,273589,2021-05-25,2021-05-25,0.9,6.160000e+02,0.000000,0.75,0.000000,0.000000,9
155,155,10-19,0.01,273589,2021-05-25,2021-06-01,0.9,5.739285e+02,573.928545,0.75,0.002098,5.739285,9
156,156,10-19,0.01,273589,2021-05-25,2021-06-08,0.9,5.127447e+02,1086.673279,0.75,0.003972,10.866733,9
157,157,10-19,0.01,273589,2021-05-25,2021-06-15,0.9,3.249209e+02,1411.594205,0.75,0.005160,14.115942,9
158,158,10-19,0.01,273589,2021-05-25,2021-06-22,0.9,1.176973e+02,1529.291460,0.75,0.005590,15.292915,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,2107,10-19,0.01,273589,2021-05-25,2021-09-21,1.7,7.406446e-06,4498.312856,0.75,0.016442,44.983129,17
2108,2108,10-19,0.01,273589,2021-05-25,2021-09-28,1.7,1.356328e-06,4498.312858,0.75,0.016442,44.983129,17
2109,2109,10-19,0.01,273589,2021-05-25,2021-10-05,1.7,2.483817e-07,4498.312858,0.75,0.016442,44.983129,17
2110,2110,10-19,0.01,273589,2021-05-25,2021-10-12,1.7,4.548565e-08,4498.312858,0.75,0.016442,44.983129,17


# Nye smittetilfælde

In [20]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

casedf = pd.DataFrame()

casedf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
casedf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


casedf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    casedf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Count.values 
    curVals = prevVals + newVals
    # casedf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Count.values 
    casedf[curAge] = curVals
    prevVals = curVals

    casedf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Count.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        casedf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 
        casedf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 


# # Remove everythings after september first
# casedf = casedf[casedf.Date <= np.datetime64('2021-09-01')]

# casedf

In [21]:
casedf

,Date,Week,Total,Total_9,Total_10,Total_11,Total_12,Total_13,Total_14,Total_15,...,90+,90+_9,90+_10,90+_11,90+_12,90+_13,90+_14,90+_15,90+_16,90+_17
0,2021-05-25,20,5580.000000,5.580000e+03,5580.000000,5580.000000,5580.000000,5580.000000,5580.000000,5580.000000,...,5580.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
1,2021-06-01,21,5967.519853,4.661934e+03,5003.674501,5334.347228,5655.273345,5967.519853,6271.964456,6569.340199,...,5967.519853,1.246604e-01,1.337985e-01,1.426407e-01,1.512223e-01,1.595718e-01,1.677127e-01,1.756645e-01,1.834439e-01,1.910651e-01
2,2021-06-08,22,6483.630044,3.956970e+03,4558.359586,5180.754876,5822.877898,6483.630044,7162.055102,7857.311782,...,6483.630044,1.553948e-02,1.790120e-02,2.034542e-02,2.286711e-02,2.546196e-02,2.812622e-02,3.085657e-02,3.365009e-02,3.650415e-02
3,2021-06-15,23,6850.656096,3.269819e+03,4041.738588,4895.811537,5832.099676,6850.656096,7951.527006,9134.752815,...,6850.656096,1.937055e-03,2.395027e-03,2.901930e-03,3.457831e-03,4.062791e-03,4.716865e-03,5.420106e-03,6.172562e-03,6.974279e-03
4,2021-06-22,24,6962.176566,2.605112e+03,3453.196134,4455.463339,5621.908474,6962.176566,8485.596119,10201.206611,...,6962.176566,2.414611e-04,3.204338e-04,4.139109e-04,5.228725e-04,6.482708e-04,7.910336e-04,9.520663e-04,1.132254e-03,1.332464e-03
5,2021-06-29,25,7150.799892,2.105970e+03,2991.052857,4106.755285,5483.141062,7150.799892,9140.741532,11484.297370,...,7150.799892,3.009902e-05,4.287125e-05,5.903733e-05,7.906562e-05,1.034400e-04,1.326589e-04,1.672348e-04,2.076933e-04,2.545727e-04
6,2021-07-06,26,6853.595977,1.597199e+03,2428.057074,3543.262522,4998.615672,6853.595977,9170.896652,12015.917436,...,6853.595977,3.751955e-06,5.735800e-06,8.420667e-06,1.195583e-05,1.650518e-05,2.224732e-05,2.937554e-05,3.809788e-05,4.863713e-05
7,2021-07-13,27,5871.429251,1.089327e+03,1770.703420,2742.758875,4081.561884,5871.429251,8203.819180,11175.850002,...,5871.429251,4.676951e-07,7.674001e-07,1.201064e-06,1.807888e-06,2.633615e-06,3.730948e-06,5.159947e-06,6.988423e-06,9.292319e-06
8,2021-07-20,28,4235.830882,6.287931e+02,1092.181855,1793.900649,2811.974594,4235.830882,6164.351607,8702.781543,...,4235.830882,5.829992e-08,1.026714e-07,1.713114e-07,2.733779e-07,4.202272e-07,6.256920e-07,9.063680e-07,1.281910e-06,1.775335e-06
9,2021-07-27,29,2509.089905,2.986066e+02,554.224867,965.023446,1592.413263,2509.089905,3796.626824,5541.078245,...,2509.089905,7.267301e-09,1.373654e-08,2.443464e-08,4.133855e-08,6.705267e-08,1.049306e-07,1.592076e-07,2.351451e-07,3.391848e-07


In [22]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
curMaxY = 30000
firstDate = np.datetime64('2021-05-18')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("cases.html",title='Nye smittetilfælde')

show(column(plot,slider))

None


In [23]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)
sourceData = ColumnDataSource(dfCaseDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
curMaxY = 30000
firstDate = np.datetime64('2021-05-18')
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("casesData.html",title='Nye smittetilfælde')

show(column(plot,slider))

None


# Hospitaliseringer

In [34]:
curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)
curdf['Hosp'] = np.multiply(np.multiply(curdf.Count,curdf.CaseHospRate),0.75)

<ipython-input-34-327fee2e2c1f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)
<ipython-input-34-327fee2e2c1f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curdf['Hosp'] = np.multiply(np.multiply(curdf.Count,curdf.CaseHospRate),0.8)


In [35]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

admdf = pd.DataFrame()

admdf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
admdf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


admdf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Hosp.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Hosp.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    admdf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Hosp.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Hosp.values 
    curVals = prevVals + newVals
    # admdf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Hosp.values 
    admdf[curAge] = curVals
    prevVals = curVals

    admdf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Hosp.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        admdf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 
        admdf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 


# # Remove everythings after september first
# admdf = admdf[admdf.Date <= np.datetime64('2021-09-01')]

# admdf

In [36]:
# dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
# dfAdm = dfAdm.transpose()
# dfAdm.columns = dfAdm.iloc[0]
# dfAdm = dfAdm.drop(['Aldersgruppe']) 
# curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')


# dfAdmDiff = dfAdm.diff().iloc[1:]

# dfAdmDiff['Dato'] = curDates[1:]

# dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
# dfAdmDiffSum['Dato'] = curDates[1:]

# dfAdmDiff = dfAdmDiff.iloc[:-1]
# dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


# dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']
# dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

In [37]:

from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 30000

firstDate = np.datetime64('2021-05-18')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

# firstDate = np.datetime64('2021-04-01')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



# for k in range(0,len(allAges)):
#     curAge = allAges[k]
#     curColor = cmaptohex(cmap(k))
#     plot.line('Dato', curAge, source=sourceData,color=curColor)
#     plot.circle('Dato', curAge, source=sourceData,color=curColor)

# plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
# plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hosp.html",title='Hospitaliseringer')

show(column(plot,slider))

None


In [38]:

from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 30000

# firstDate = np.datetime64('2021-05-18')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hospData.html",title='Hospitaliseringer')

show(column(plot,slider))

None


# OLD

In [18]:

# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color='#00ff00')
# plot.line('Week', '20-29', source=source, line_width=3, line_alpha=0.6, color='#20dd20')
# plot.line('Week', '30-39', source=source, line_width=3, line_alpha=0.6, color='#307030')
# plot.line('Week', '40-49', source=source, line_width=3, line_alpha=0.6, color='#406040')
# plot.line('Week', '50-59', source=source, line_width=3, line_alpha=0.6, color='#505050')
# plot.line('Week', '60-69', source=source, line_width=3, line_alpha=0.6, color='#604060')
# plot.line('Week', '70-79', source=source, line_width=3, line_alpha=0.6, color='#703070')
# plot.line('Week', '80-89', source=source, line_width=3, line_alpha=0.6, color='#802080')
# plot.line('Week',   '90+', source=source, line_width=3, line_alpha=0.6, color='#901090')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6,color='black')

# plot.circle('Week', '10-19', source=source, color='#108000')
# plot.circle('Week', '20-29', source=source, color='#208000')
# plot.circle('Week', '30-39', source=source, color='#308000')
# plot.circle('Week', '40-49', source=source, color='#408000')
# plot.circle('Week', '50-59', source=source, color='#508000')
# plot.circle('Week', '60-69', source=source, color='#608000')
# plot.circle('Week', '70-79', source=source, color='#708000')
# plot.circle('Week', '80-89', source=source, color='#808000')
# plot.circle('Week',   '90+', source=source, color='#908000')
# plot.circle('Week', 'Total', source=source,color='black')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6)


# plot.varea('Week', 0 ,'10-19', source=source, fill_color=cmaptohex(cmap(1)))
# plot.varea('Week','10-19', '20-29', source=source, color=cmaptohex(cmap(2)))
# plot.varea('Week','20-29', '30-39', source=source, color=cmaptohex(cmap(3)))
# plot.varea('Week','30-39', '40-49', source=source, color=cmaptohex(cmap(4)))
# plot.varea('Week','40-49', '50-59', source=source, color=cmaptohex(cmap(5)))
# plot.varea('Week','50-59', '60-69', source=source, color=cmaptohex(cmap(6)))
# plot.varea('Week','60-69', '70-79', source=source, color=cmaptohex(cmap(7)))
# plot.varea('Week','70-79', '80-89', source=source, color=cmaptohex(cmap(8)))
# plot.varea('Week','80-89',   '90+', source=source, color=cmaptohex(cmap(9)))

# slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

In [19]:
# curdf['RT2'] = np.round(np.multiply(curdf.RT,10)).astype(int)


    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i]
    #     y3[i] = data[curName3][i]
    #     y4[i] = data[curName4][i]
    #     y5[i] = data[curName5][i]
    #     y6[i] = data[curName6][i]
    #     y7[i] = data[curName7][i]
    #     y8[i] = data[curName8][i]
    #     y9[i] = data[curName9][i]
    #     yT[i] = data[curNameT][i]
    # }


    # // Summing up
    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i] + y1[i]
    #     y3[i] = data[curName3][i] + y2[i]
    #     y4[i] = data[curName4][i] + y3[i]
    #     y5[i] = data[curName5][i] + y4[i]
    #     y6[i] = data[curName6][i] + y5[i]
    #     y7[i] = data[curName7][i] + y6[i]
    #     y8[i] = data[curName8][i] + y7[i]
    #     y9[i] = data[curName9][i] + y8[i]
    #     yT[i] = data[curNameT][i]
    # }

In [20]:

# # x = [x*0.005 for x in range(0, 201)] 

# # source = ColumnDataSource(data=dict(x=x, y=x))
# # source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
# source = ColumnDataSource(curdf)

# plot = figure(plot_width=400, plot_height=400)
# plot.line('x', 'Count', source=source, line_width=3, line_alpha=0.6)

# # slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")
# minRT = 0.9
# maxRT = 1.7
# slider = Slider(start=minRT, end=maxRT, value=1.1, step=.1, title="RT")

# update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
#     var data = source.data;
#     var f = slider.value;
#     var age = data['Age']
#     var RT = data['RT2']
#     var End = data['End']
#     var Count = data['Count']

#     var x = data['x']
#     var y = data['y']

#     console.log(Count)
#     for (var i = 0; i < RT.length; i++) {
#         y[i] = 6
#         Count[i] = 0
        
#         if (RT[i] == Math.round(f*10)){
#             y[i] = data['Count'][i]
#             console.log(data['Count'][i])
#             Count[i] = data['Count'][i]

#             // End.push(data['End'][i])
#             // Count.push(data['Count'][i])
#         }

#     }

#     //for (var i = 0; i < x.length; i++) {
#         // y[i] = Math.pow(x[i], f)
#     //}
    
#     // necessary becasue we mutated source.data in-place
#     source.change.emit();
# """)
# slider.js_on_change('value', update_curve)

# output_file("plot.html")

# show(column(slider, plot))

In [21]:

# testdf['Total'] = curdf[(curdf.RT2 == 11)].Count.values 
# for curRT in allRTs:
#     curColName = 'Total_'+str(curRT)
#     testdf[curColName] = curdf[(curdf.RT2 == curRT)].Count.values 

# testdf['20_RT10'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 10)].Count.values
# testdf['20_RT11'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['20_RT12'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['20_RT13'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['20_RT14'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values
# testdf['RT11_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['RT12_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['RT13_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['RT14_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values

    # if (Math.round(f*10) == 11){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT11_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 12){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT12_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 13){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT13_20'][i]
    #     }
    # }
    
